In [ ]:
import pyvista as pv
from pathlib import Path

path_base = Path("../pebble_cht")

# get_reader function uses the file extensions to determine which reader should be used
fluid_reader = pv.get_reader(path_base / "pebble.nek5000")

fluid_reader.enable_spectral_element_ids()
fluid = fluid_reader.read()

print(fluid.cell_data)

3D-Mesh found, spectral element of size = 4*4*4=64
pyvista DataSetAttributes
Association     : CELL
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    spectral element id     uint32     (51408,)


In [3]:
import numpy as np

def np_array(ordering):
    """Wrapper for np.array to simplify common modifications"""
    return np.array(ordering, dtype=np.float64)

def n_verts_between(n, frm, to):
    """Places `n` vertices on the edge between `frm` and `to`"""
    if n <= 0:
        return np.ndarray((0, 3)) # empty
    edge_verts = np.stack((
        np.linspace(frm[0], to[0], num=n+1, endpoint=False, axis=0),   # n+1 since start is included, remove later
        np.linspace(frm[1], to[1], num=n+1, endpoint=False, axis=0),
        np.linspace(frm[2], to[2], num=n+1, endpoint=False, axis=0),
        ), axis=1)
    return edge_verts[1:] # remove start point

def sort_by_axes(coords):
    coords = coords.round(12) # TODO required to some extent to get sorting right, better way to do this?
    reordering = np.lexsort((coords[:,0], coords[:,1], coords[:,2]))
    return coords[reordering, :]

def number_quadrilateral(corner_verts, order, skip=False):
    """Outputs the list of coordinates of a right-angled quadrilateral of arbitrary order in the right ordering"""
    # first: corner vertices
    coords = np_array(corner_verts) if not skip else np.ndarray((0, 3)) # empty if skip
    # second: edges
    num_verts_on_edge = order -1
    edges = [(0,1), (1,2), (3,2), (0,3)]
    for frm, to in edges:
        coords = np.concatenate([coords, n_verts_between(num_verts_on_edge, corner_verts[frm], corner_verts[to])], axis=0) if not skip else np.ndarray((0, 3)) # empty if skip
    # third: face
    e_x = (corner_verts[1] - corner_verts[0]) / order
    e_y = (corner_verts[3] - corner_verts[0]) / order
    pos_y = corner_verts[0].copy()
    pos_y = np.expand_dims(pos_y, axis=0)
    for _ in range(num_verts_on_edge):
        pos_y += e_y
        pos_yx = pos_y.copy()
        for _ in range(num_verts_on_edge):
            pos_yx += e_x
            coords = np.concatenate([coords, pos_yx], axis=0)
    return coords


def number_hexahedron(corner_verts, order):
    """Outputs the list of coordinates of a right-angled hexahedron of arbitrary order in the right ordering"""
    # first: corner vertices
    coords = np_array(corner_verts)
    # second: edges
    num_verts_on_edge = order - 1
    edges = [(0,1), (1,2), (3,2), (0,3), (4,5), (5,6), (7,6), (4,7), (0,4), (1,5), (2,6), (3,7)] # TODO: not as in documentation, beware of future changes!!
    for frm, to in edges:
        coords = np.concatenate([coords, n_verts_between(num_verts_on_edge, corner_verts[frm], corner_verts[to])], axis=0)
    # third: faces
    faces = [(0,3,7,4), (1,2,6,5), (0,1,5,4), (3,2,6,7), (0,1,2,3), (4,5,6,7)] # TODO: not as in documentation, beware of future changes!!
    for indices in faces:
        sub_corner_verts = [corner_verts[q] for q in indices]
        face_coords = number_quadrilateral(np_array(sub_corner_verts), order, skip=True) # use number_quadrilateral to number face, but skip cornes and edges
        coords = np.concatenate([coords, face_coords], axis=0)
    # fourth: interior
    e_x = (corner_verts[1] - corner_verts[0]) / order
    e_y = (corner_verts[3] - corner_verts[0]) / order
    e_z = (corner_verts[4] - corner_verts[0]) / order
    pos_z = corner_verts[0].copy()
    pos_z = np.expand_dims(pos_z, axis=0)
    for _ in range(num_verts_on_edge):
        pos_z += e_z
        pos_zy = pos_z.copy()
        for _ in range(num_verts_on_edge):
            pos_zy += e_y
            pos_zyx = pos_zy.copy()
            for _ in range(num_verts_on_edge):
                pos_zyx += e_x
                coords = np.concatenate([coords, pos_zyx], axis=0)
    return coords.astype('i4')

def reorder_points(n):

    corner_verts = np.array([[0,0,0], [n,0,0],[n,n,0], [0,n,0], [0,0,n], [n,0,n], [n,n,n], [0,n,n]], 'f4')
    coords = number_hexahedron(corner_verts, n)
    npoints = n + 1
    return np.array([x + y*npoints + npoints*npoints*z for x, y, z in coords])


print(reorder_points(3))


[ 0  3 15 12 48 51 63 60  1  2  7 11 13 14  4  8 49 50 55 59 61 62 52 56
 16 32 19 35 31 47 28 44 20 24 36 40 23 27 39 43 17 18 33 34 29 30 45 46
  5  6  9 10 53 54 57 58 21 22 25 26 37 38 41 42]


In [ ]:
import vtk

N_elements = fluid.cell_data['spectral element id'].max()
print(fluid.n_cells, N_elements)
new_grid = pv.UnstructuredGrid()

new_points = vtk.vtkPoints()
hex_array = vtk.vtkCellArray()

order = 3
scalars = fluid.point_data.keys()
index_mapper = reorder_points(order)
points_per_element = (order+1)*(order+1)*(order+1)
point_data = []
for key in fluid.point_data.keys():
    shape = N_elements*points_per_element if fluid.point_data[key].ndim == 1 else (N_elements*64, fluid.point_data[key].shape[-1])
    point_data.append(np.zeros(shape))

for i in range(N_elements):
    element = fluid.extract_values(values=i,
                                   preference='cell',
                                   scalars='spectral element id')
    hex = vtk.vtkLagrangeHexahedron()
    hex.GetPointIds().SetNumberOfIds(element.n_points)
    hex.GetPoints().SetNumberOfPoints(element.n_points)
    for k, scalar in enumerate(scalars):
        start = i*element.n_points
        end = (i+1)*element.n_points
        point_data[k][start:end] = element.point_data[scalar][index_mapper]

    for j in range(element.n_points):

        hex.GetPointIds().SetId(j, i*element.n_points + j)
        hex.GetPoints().SetPoint(j, *element.points[index_mapper[j]])
        new_points.InsertNextPoint(element.points[index_mapper[j]])
    
    hex.SetUniformOrderFromNumPoints(element.n_points)

    hex_array.InsertNextCell(hex)

new_grid.SetPoints(new_points)
new_grid.SetCells(pv.CellType.LAGRANGE_HEXAHEDRON, hex_array)

for i, scalar in enumerate(scalars):
    new_grid.point_data[scalar] = point_data[i]


print(new_grid)

51408 1903
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3

KeyboardInterrupt: 

In [24]:
new_grid.clip('x',(0,0,0)).plot(cmap='bwr',scalars='Temperature')

Widget(value='<iframe src="http://localhost:35355/index.html?ui=P_0x7ff8055930d0_9&reconnect=auto" class="pyvi…